# Inference time measurement 

To be able to run this experimentation, [Grid2op](https://github.com/rte-france/Grid2Op) (1.7.1) and [LightSim2Grid](https://github.com/BDonnot/lightsim2grid) (0.7.0) are required. See the corresponding links for more information concerning their installation.

In [ ]:
import pathlib
import numpy as np
from pprint import pprint
from matplotlib import pyplot as plt
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

# TOC
- [Device Selection](#device)
- [Environment: `l2rpn_case14_sandbox`](#env_case14)
    - [Benchmark Selection](#benchmark_select)
      - [Benchmark1](#benchmark1)
          - [Physics solver time](#14_solver_bench1)
      - [Benchmark2](#benchmark2)
          - [Physics solver time](#14_solver_bench2)
- [Environment: `l2rpn_neurips_2020_track1_small`](#env_nips)
    - [Benchmark1](#nips_benchmark1)
        - [Physics solver time](#nips_solver_bench1)
    - [Benchmark2](#nips_benchmark2)
        - [Physics solver time](#nips_solver_bench2)

## Select a device on which the model should be evaluated <a id="device"></a>
On the basis of GPU or CPU device, the results may be vary significantly. You can choose one of the following block in each kernel to using whether GPU or CPU device. 

### Using A GPU

In [ ]:
import tensorflow as tf
device = "GPU"
gpus = tf.config.experimental.list_physical_devices(device)
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[3], device)
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

### Using CPU

In [ ]:
import os
device = "CPU"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

### Show the list of used devices

In [ ]:
tf.config.experimental.get_visible_devices()

# Environment : "l2rpn_case14_sandbox" <a id="env_case14"></a>

In [ ]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Select a benchmark <a id="benchmark_select"></a>

### Benchmark1 <a id="benchmark1"></a>

In [ ]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{3}_\textrm{# situations} \times \underbrace{6}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{20}$$, where $20$ is number of the lines in this benchmark. In our case the *security analysis* takes 0.02 ms to be solved. So the solver time is $3 \times 6 \times \frac{0.02}{20} = 0.018 ms$

In [ ]:
physics_solver_time = 0.018

### Select a model 

#### Fully connected model

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

##### Load it from a saved model

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

##### Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

#### LeapNet model 

In [ ]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark1.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["time_inf"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["time_inf"])

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

##### Computing the speed up

In [ ]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (0.36 / inf_time_ML)))

## Benchmark2 <a id="benchmark2"></a>

In [ ]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 10
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [ ]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {} s".format(NB_RUN, physics_solver_time))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms".format(NB_RUN, (physics_solver_time/10000)*1000))

In [ ]:
physics_solver_time = 1.9429454155266286

### Select a model

#### Fully Connected model

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

##### Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

#### LeapNet model 

In [ ]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark2.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["time_inf"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["time_inf"])

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

##### Computing speed up

In [ ]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using LeapNet model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

# Environment : "l2rpn_neurips_2020_track1_small" <a id="env_nips"></a>

In [ ]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Benchmark1 <a id="nips_benchmark1"></a>

In [ ]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{3}_\textrm{# situations} \times \underbrace{6}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{36}$$, where $36$ is number of the lines for this environment. In our case the *security analysis* takes 0.02 ms to be solved. So the solver time is $3 \times 6 \times \frac{0.03}{36} = 0.015 ms$

In [ ]:
physics_solver_time = 0.015

### Select a model 

#### Fully connected model

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

##### Load it from a saved model

In [ ]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f} ms".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {:.2f} ms".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = (np.mean(time_ind) / 10000) * 1000
inf_time_ML = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {:.2f}".format(device, inf_time_ML))

##### Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

## Benchmark2 <a id="nips_benchmark2"></a>

In [ ]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 10
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [ ]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {} s".format(NB_RUN, physics_solver_time))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms".format(NB_RUN, (physics_solver_time/10000)*1000))

In [ ]:
physics_solver_time = 2.8298422046704217

### Select a model

#### Fully Connected model

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(10):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["time_inf"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["time_inf"])

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

In [ ]:
inf_time_IndRed = np.mean(time_ind)
inf_time_ML = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {}".format(device, inf_time_IndRed))
print("Inference time from Machine Learning category and using {}: {}".format(device, inf_time_ML))

##### Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML)))